In [69]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [70]:
df = pd.read_csv('imdb_master.csv', encoding = "ISO-8859-1")
del df['Unnamed: 0']
del df['file']
del df['type']

In [71]:
df = df.loc[0:49999]
df = df.sample(frac=1).reset_index(drop=True)

In [72]:
def clean_reviews(no_reviews,dataset):
    import re
    import nltk
    nltk.download('stopwords') 
    from nltk.corpus import stopwords 
    
    my_stopwords = stopwords.words('english') 
            
    from nltk.stem.porter import PorterStemmer
     
    corpus =[] 
    for i in range (0, no_reviews):
        review = re.sub('[^a-zA-Z]',' ', dataset['review'][i]) #Removes all other characters besides the letters and puts the review in a string
        review = review.lower() 
        review = review.split()
        ps = PorterStemmer() 
        review = [ps.stem(word) for word in review if not word in set (my_stopwords)]
        review = ' '.join(review) 
        corpus.append(review)
    return corpus

In [73]:
x = clean_reviews(50000,df)

[nltk_data] Downloading package stopwords to /home/aayush/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [74]:
def bag_of_words(corpus, maximum_features):
    from sklearn.feature_extraction.text import CountVectorizer
    cv = CountVectorizer(max_features = maximum_features)
    X = cv.fit_transform(corpus)
    return X

In [75]:
x

['okay movi went straight video paid see would disappoint morn alon night pretti good fright hey rhyme br br',
 'one go mistak squall good movi sure memor one taken myrna loy perform nubi hot blood gypsi girl like forget experi film made exot beauti miss loy still cast foreign vixen often asian usual sinist certainli eye appear skin darken hair curl scene barefoot wear littl skirt loos fit peasant blous one scene wear noth pattern towel suppos focus miss loy appear far best reason tune creaki antiqu keep watch sure attract dialogu hopeless one typic passag nubi gaze window depart caravan wax poetic alway gypsi sing weird sad big sun breath fire burn pale moon look behind cloud breath air cold death sing poetic lover purpl prose field day help wonder though later year miss loy prefer recal involv project br br like mani earli talki one adapt recent broadway success stage version open th street theatr novemb ran year play provok famou episod involv humorist theater critic robert benchley

In [76]:
x_data = bag_of_words(x,1000)

In [77]:
x_data

<50000x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 3029044 stored elements in Compressed Sparse Row format>

In [78]:
labels = df['label']
labels = labels.map({'pos': 1, 'neg': 0})

In [79]:
train_data = x_data[0:34999,:]
test_data = x_data[35000:49999,:]
train_labels = labels[0:34999]
test_labels = labels[35000:49999]

In [80]:
from sklearn.svm import LinearSVC
svm = LinearSVC()

In [81]:
svm.fit(train_data, train_labels)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [82]:
y_pred_class = svm.predict(test_data)

In [83]:
from sklearn import metrics
metrics.accuracy_score(test_labels, y_pred_class)

0.8621908127208481

In [84]:
metrics.confusion_matrix(test_labels, y_pred_class)

array([[6318, 1155],
       [ 912, 6614]])